In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import scipy as sci
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 1000)

In [6]:
filepath = 'C:/Users/night/Documents/Out of the Park Developments/OOTP Baseball 23/saved_games/SBC.lg/import_export/csv/'

In [34]:
players = pd.read_csv(filepath + 'players.csv', low_memory = False)
batters_stats = pd.read_csv(filepath + 'players_career_batting_stats.csv')
pitchers_stats = pd.read_csv(filepath + 'players_career_pitching_stats.csv')
teams = pd.read_csv(filepath + 'teams.csv')

In [35]:
teams = teams[(teams['league_id'] == 145) & (teams['team_id'] <= 60)]
teams['team_name'] = teams['name'] + ' ' + teams['nickname']
teams = teams[['team_id','team_name']]

In [36]:
undrafted = players.loc[(players['draft_year'] == 0) & (players['age'] < (max(batters_stats['year']) - 2016)), ('player_id', 'team_id', 'position', 'role', 'first_name', 'last_name')]

In [24]:
batters = batters_stats.loc[batters_stats['level_id'] == 1,['player_id', 'year', 'team_id','war']]
batters = batters.groupby(['player_id', 'year', 'team_id']).sum().reset_index()
batters.rename(columns={'war':'batter_war'}, inplace = True)
batters.count()

player_id     126339
year          126339
team_id       126339
batter_war    126339
dtype: int64

In [25]:
pitchers = pitchers_stats.loc[pitchers_stats['level_id'] == 1,['player_id', 'year', 'team_id','war']]
pitchers = pitchers.groupby(['player_id', 'year', 'team_id']).sum().reset_index()
pitchers.rename(columns={'war':'pitcher_war'}, inplace = True)
pitchers.count()

player_id      61410
year           61410
team_id        61410
pitcher_war    61410
dtype: int64

In [26]:
eligible_players = pd.merge(batters,pitchers, how = 'outer', on = ('player_id', 'team_id', 'year')).fillna(0)
eligible_players['war'] = round(eligible_players['batter_war'] + eligible_players['pitcher_war'],2)
eligible_players = eligible_players[(eligible_players['team_id'] <= 60) & (eligible_players['team_id'] > 29)]
eligible_players = eligible_players[['player_id', 'year','team_id','war']]
eligible_players

,player_id,year,team_id,war
0,5,2022,53,-0.58
1,5,2022,56,-0.25
2,5,2023,53,0.15
3,7,2015,35,1.79
4,7,2016,35,2.38
...,...,...,...,...
134480,51391,2046,41,2.21
134481,51471,2043,38,-0.52
134482,51471,2044,38,0.79
134483,51471,2045,38,0.85


In [27]:
undrafted_stats = pd.merge(eligible_players, undrafted,on=('player_id','team_id')).merge(teams, on = 'team_id')

In [28]:
undrafted_stats['position'] = np.where(undrafted_stats['position'] == 2, 'C',
                              np.where(undrafted_stats['position'] == 3, '1B',
                              np.where(undrafted_stats['position'] == 4, '2B',
                              np.where(undrafted_stats['position'] == 5, '3B',
                              np.where(undrafted_stats['position'] == 6, 'SS',
                              np.where(undrafted_stats['position'] == 7, 'LF',
                              np.where(undrafted_stats['position'] == 8, 'CF',
                              np.where(undrafted_stats['position'] == 9, 'RF',
                              np.where(undrafted_stats['position'] == 10, 'DH',
                              np.where((undrafted_stats['position'] == 1) & (undrafted_stats['role'] == 11), 'SP',
                              np.where((undrafted_stats['position'] == 1) & (undrafted_stats['role'] == 12), 'RP',
                              np.where((undrafted_stats['position'] == 1) & (undrafted_stats['role'] == 13), 'CL',''))))))))))))

In [37]:
undrafted_stats['player_name'] = undrafted_stats['first_name'] + ' ' + undrafted_stats['last_name']
final_cols=['player_name', 'team_name', 'position','war']
undrafted_stats_final = undrafted_stats[final_cols]
undrafted_stats_final

,player_name,team_name,position,war
0,Leith Commo,Cleveland Spiders,3B,-0.21
1,Leith Commo,Cleveland Spiders,3B,3.99
2,Leith Commo,Cleveland Spiders,3B,3.99
3,Leith Commo,Cleveland Spiders,3B,5.14
4,Leith Commo,Cleveland Spiders,3B,2.88
5,Leith Commo,Cleveland Spiders,3B,3.95
6,Leith Commo,Cleveland Spiders,3B,4.16
7,Leith Commo,Cleveland Spiders,3B,1.98
8,Luis Medel,Cleveland Spiders,2B,2.91
9,Luis Medel,Cleveland Spiders,2B,1.08


In [30]:
final_undrafted = undrafted_stats_final.groupby('team_name').sum().sort_values('war', ascending = False)

In [31]:
final_undrafted['Z-Score'] = round((final_undrafted['war'] - final_undrafted['war'].mean())/final_undrafted['war'].std(),2)

In [32]:
final_undrafted

,war,Z-Score
team_name,,
Cleveland Spiders,77.22,2.64
St. Louis Cardinals,70.63,2.31
Chicago Cubs,57.68,1.67
Boston Red Sox,53.48,1.46
San Francisco Giants,43.40,0.96
Golden State Grizzlies,37.17,0.65
Washington Nationals,35.74,0.58
Florida Marlins,32.98,0.44
Minnesota Twins,31.67,0.38
